# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [75]:
import pandas as pd
import seaborn as sn
Data = pd.read_excel("tweets_rappi.xlsx")
relevante = Data.Treinamento[Data.Classifcação == 'relevante']
irrelevante = Data.Treinamento[Data.Classifcação == 'irrelevante']
#FUNCAO PARA LIMPEZA PARA EVITAR CODIGO EXTENSO#
def LimpezaPrevia(tweets):
    acentos = {'á':'a','ã':'a','é':'e','ê':'e','í':'i','ô':'o','ó':'o','ú':'u','õ':'o'}
    words = []
    for i in tweets:
        i.replace('/n','')
        i.replace('!','')
        i.replace("'","")
        lista = i.split(" ")
        for x in lista:
            if '@' in x:
                del x
            elif 'http' in x:
                del x
            elif x == 'rt':
                  del x
            elif '\n' in x:
                del x
            else:
                z = list(x)
                for h in z:
                    if h == '?':
                        z.remove(h)
                    elif h == ',':
                        z.remove(h)
                    elif h == '.':
                        z.remove(h)
                    elif h == '!':
                        z.remove(h)
                    elif h == '#':
                        z.remove(h)
                    elif h == '&':
                        z.remove(h)
                    elif h == ';':
                        z.remove(h)
                    elif h == '(':
                        z.remove(h)
                    elif h == ')':
                        z.remove(h)
                    elif h == '"':
                        z.remove(h)
                    elif h == '/':
                        z.remove(h)
                    elif h == '*':
                        z.remove(h)
                    elif h == '-':
                        z.remove(h)
                    elif h in acentos:#RETIRANDO ACENTOS MAIS USADOS#
                        w = acentos[h]
                        z.remove(h)
                        z.append(w)
                y = ''.join(z)
                words.append(y)
    return words

def LimpezaLinha(lista):#FUNCAO LIMPEZA PARA UMA LINHA, USADA SOMENTE PARA TESTE E APREDIZADO MEU#
    lista2 = []
    for x in lista:
        if '@' in x:
            del x
        elif 'http' in x:
            del x
        elif x == 'rt':
            del x
        elif '\n' in x:
            del x
        else:
            z = list(x)
            for h in z:
                if h == '?':
                    z.remove(h)
                elif h == ',':
                    z.remove(h)
                elif h == '.':
                    z.remove(h)
                elif h == '!':
                    z.remove(h)
                elif h == '#':
                    z.remove(h)
                elif h == '&':
                    z.remove(h)
                elif h == ';':
                    z.remove(h)
                elif h == '(':
                    z.remove(h)
                elif h == ')':
                    z.remove(h)
                elif h == '"':
                    z.remove(h)
                elif h == '/':
                    z.remove(h)
                elif h == '*':
                    z.remove(h)
                elif h == '-':
                    z.remove(h)
                elif h in acentos:
                    w = acentos[h]
                    z.remove(h)
                    z.append(w)
            y = ''.join(z)
            lista2.append(y)
    return lista2
#Montando Series com palavras limpas e separadas#
#Dicionario com palavras relevantes e suas respectivas probabilidades#
palavrasRel = LimpezaPrevia(relevante)
Relevantes = pd.Series(palavrasRel)
PRel = dict(Relevantes.value_counts(normalize=True))

#Dicionario com palavras irrelevantes e suas respectivas probabilidades#
palavrasIrrel = LimpezaPrevia(irrelevante)
Irrelevantes = pd.Series(palavrasIrrel)
PIrrel = dict(Irrelevantes.value_counts(normalize=True))


#IGNORAR DAQUI PARA BAIXO, NAO FAZ PARTE DO CODIGO FINAL#
#---------------------------------------------------------------------------#
#Algoritmo de Classificação#
#TESTE DE LINHAS PARA TENTAR ENTENDER O NAIVE BAYES CLASSFIER, POR FAVOR IGNORAR#
Datateste = pd.read_excel('tweets_rappi_teste.xlsx')#TWEETS TESTE#
tweet1 = Datateste.iloc[114,0] #PEGANDO UMA LINHA#
palavrasteste = str(tweet1).split(" ")
testefinal = LimpezaLinha(palavrasteste)
palavrasteste
PTesteRel = 1
PTesteIrrel = 1
for i in testefinal:
    if i in PRel:
        PTesteRel = PTesteRel*PRel[i]
    else:
        PTesteRel = PTesteRel *1
print(PTesteRel)
for i in testefinal:
    if i in PIrrel:
        PTesteIrrel = PTesteIrrel * PIrrel[i]
    else:
        PTesteIrrel = PTesteIrrel * 1
print(PTesteIrrel)
print(tweet1)
if PTesteRel > PTesteIrrel:
    print("RELEVANTE")
else:
    print("IRRELEVANTE")

3.083529679274882e-51
1.251887286390027e-68
quem quer comer de graça hoje? 

com o glovo, seu primeiro pedido é por nossa conta usando o cupom de desconto 179m4ed.

é melhor que ifood, rappi e uber eats, beleza?

experimente esse xuxuzinho de app ❤ https://t.co/pamjgkf4x3
RELEVANTE


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
